# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.multioutput import MultiOutputClassifier
import pickle

import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import os
os.getcwd()

'/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Disaster'

In [2]:
path = Path('/Users/yejiseoung/Dropbox/My Mac (Yejis-MacBook-Pro.local)/Documents/Projects/Disaster')

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster.db')
df = pd.read_sql('SELECT * FROM disaster', engine)
df.head(2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [4]:
df.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [5]:
df['child_alone'].unique()

array([0])

In [6]:
df['related'].unique()

array([1, 0, 2])

In [7]:
X = df.filter(items=['id', 'message', 'original', 'genre'])
y = df.drop(['id', 'message', 'original', 'genre', 'child_alone'], axis=1) # child_alone has all '0' values

# Change '2' values in 'related' to '1'
y['related'] = y['related'].map(lambda x: 1 if x==2 else x)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X['message'].iloc[[1]]

1    Is the Hurricane over or is it not over
Name: message, dtype: object

In [9]:
X['original'].iloc[[1]]

1    Cyclone nan fini osinon li pa fini
Name: original, dtype: object

### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(w).lower().strip() for w in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
    ('cvect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
pipeline.fit(X_train['message'], y_train)

Pipeline(steps=[('cvect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ffc68be0e50>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
y_pred_test = pipeline.predict(X_test['message'])
y_pred_train = pipeline.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('-----------------------------------------------------------------------------------')
print('\n',classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.96      0.88      5992
               request       0.90      0.42      0.58      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.82      0.49      0.61      3219
          medical_help       0.50      0.00      0.01       638
      medical_products       0.20      0.00      0.00       418
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       144
              military       0.00      0.00      0.00       245
                 water       0.94      0.13      0.23       500
                  food       0.95      0.23      0.37       878
               shelter       0.87      0.10      0.18       705
              clothing       0.67      0.02      0.03       115
                 money       0.86      0.04      0.07       170
        missing_people       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
parameters = {'clf__n_estimators': [10, 20, 40],
              'clf__max_depth': [10, 20, None],
             'clf__min_samples_leaf': [1, 2, 4],
             'clf__min_samples_split': [2, 5, 10],
             }

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='f1_micro', verbose=1, n_jobs=1)
cv.fit(X_train['message'], y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed: 40.3min finished


GridSearchCV(estimator=Pipeline(steps=[('cvect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ffc68be0e50>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             n_jobs=1,
             param_grid={'clf__max_depth': [10, 20, None],
                         'clf__min_samples_leaf': [1, 2, 4],
                         'clf__min_samples_split': [2, 5, 10],
                         'clf__n_estimators': [10, 20, 40]},
             scoring='f1_micro', verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred_test = cv.predict(X_test['message'])
y_pred_train = cv.predict(X_train['message'])
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))
print('--------------------------------------------------------------------------------')
print('\n', classification_report(y_train.values, y_pred_train, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5992
               request       0.85      0.41      0.55      1332
                 offer       0.00      0.00      0.00        36
           aid_related       0.77      0.50      0.61      3219
          medical_help       0.70      0.01      0.02       638
      medical_products       0.57      0.01      0.02       418
     search_and_rescue       0.00      0.00      0.00       192
              security       0.00      0.00      0.00       144
              military       0.71      0.02      0.04       245
                 water       0.93      0.17      0.28       500
                  food       0.89      0.22      0.35       878
               shelter       0.82      0.10      0.19       705
              clothing       0.80      0.03      0.07       115
                 money       0.86      0.04      0.07       170
        missing_people       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
cv.best_params_

{'clf__max_depth': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 5,
 'clf__n_estimators': 20}

### 9. Export your model as a pickle file

In [17]:
random_m1 = pickle.dumps('clf')

In [18]:
# load the model
m = pickle.loads(random_m1)

In [19]:
m

'clf'

In [20]:
ramdom_m1 = pickle.dump('clf', open('random_m1.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.